# Chapter 28: Securing Your Applications

Security is not an afterthought – it's a critical aspect of software development. A single vulnerability can lead to data breaches, financial loss, and reputational damage. As a developer, you must understand common threats and how to protect your applications and users.

In this chapter, you'll learn:

- **Common vulnerabilities** – SQL injection, cross‑site scripting (XSS), cross‑site request forgery (CSRF), and more.
- How to **protect secrets** – using User Secrets, Azure Key Vault, and environment variables.
- **Hashing and salting passwords** – why you should never store plaintext passwords.
- **Authentication** – verifying user identity, including JWT (JSON Web Tokens) and ASP.NET Core Identity.
- **Authorization** – controlling access to resources.
- **HTTPS** and secure communication.
- Security best practices for APIs and web applications.
- A practical example: adding authentication and authorization to the Notes API from Chapter 22.

By the end, you'll be equipped to build applications that are resilient against common attacks and follow industry‑standard security practices.

---

## 28.1 Common Web Application Vulnerabilities

Understanding the threats is the first step. The **OWASP Top 10** is a standard awareness document for web application security. Here are some of the most critical vulnerabilities you must guard against.

### 28.1.1 SQL Injection

SQL injection occurs when untrusted data is sent to an interpreter as part of a command or query. An attacker can craft input to modify the SQL query, potentially retrieving, modifying, or deleting data.

**Vulnerable code:**

```csharp
string query = "SELECT * FROM Users WHERE UserName = '" + userName + "'";
```

If `userName` is `' OR '1'='1`, the query becomes `SELECT * FROM Users WHERE UserName = '' OR '1'='1'`, returning all users.

**Prevention:**

- Use **parameterized queries** or an ORM like Entity Framework Core.
- Never concatenate user input directly into SQL strings.

```csharp
// With EF Core
var user = context.Users.FirstOrDefault(u => u.UserName == userName);
```

### 28.1.2 Cross‑Site Scripting (XSS)

XSS allows attackers to inject malicious scripts into web pages viewed by other users. This can lead to session hijacking, defacement, or redirection.

**Prevention:**

- **Encode output** – always encode user‑supplied data before rendering it in HTML. In Razor, the default `@` syntax encodes automatically.
- Use **Content Security Policy (CSP)** headers to restrict where scripts can be loaded from.
- Validate and sanitize input, but encoding is more reliable.

### 28.1.3 Cross‑Site Request Forgery (CSRF)

CSRF forces a logged‑in user to execute unwanted actions on a web application. For example, an attacker could trick a user into submitting a form that changes their email address.

**Prevention:**

- Use **anti‑forgery tokens** (e.g., `[ValidateAntiForgeryToken]` in ASP.NET Core MVC).
- For APIs, use token‑based authentication and ensure that requests include the token (e.g., JWT in headers), which is not automatically sent by browsers.

### 28.1.4 Other Vulnerabilities

- **Insecure Direct Object References (IDOR)** – exposing internal object IDs without authorization checks. Always verify that the user is allowed to access the requested resource.
- **Security Misconfiguration** – default credentials, verbose error messages, unnecessary features. Harden your configuration.
- **Sensitive Data Exposure** – not encrypting sensitive data in transit (use HTTPS) or at rest (encrypt passwords, personal data).

---

## 28.2 Protecting Secrets

Never hard‑code secrets (passwords, API keys, connection strings) in source code. They will end up in version control and can be exposed.

### 28.2.1 Environment Variables

Store secrets in environment variables specific to the deployment environment.

```csharp
string connectionString = Environment.GetEnvironmentVariable("DB_CONNECTION");
```

### 28.2.2 User Secrets (Development)

In development, use the **Secret Manager** tool to store secrets outside the project tree.

```bash
dotnet user-secrets init
dotnet user-secrets set "DbPassword" "mySecret123"
```

Access via configuration:

```csharp
var password = builder.Configuration["DbPassword"];
```

### 28.2.3 Azure Key Vault (Production)

For production, use a dedicated secrets management service like Azure Key Vault. It provides secure storage, access policies, and auditing.

```csharp
builder.Configuration.AddAzureKeyVault(
    new Uri("https://myvault.vault.azure.net/"),
    new DefaultAzureCredential());
```

---

## 28.3 Hashing and Salting Passwords

Storing passwords in plaintext is unforgivable. If your database is compromised, all user passwords are exposed. Always hash and salt passwords.

### Hashing vs. Encryption

- **Hashing** is a one‑way function. You cannot reverse a hash to get the original password.
- **Encryption** is two‑way – you can decrypt if you have the key. Not suitable for passwords.

### Salting

A **salt** is a random value added to the password before hashing. It ensures that even if two users have the same password, their hashes differ. Salts also prevent the use of precomputed rainbow tables.

### Using `Rfc2898DeriveBytes` (PBKDF2)

.NET provides `Rfc2898DeriveBytes` for password hashing.

```csharp
public static string HashPassword(string password, out string salt)
{
    byte[] saltBytes = new byte[16];
    using (var rng = RandomNumberGenerator.Create())
    {
        rng.GetBytes(saltBytes);
    }

    var pbkdf2 = new Rfc2898DeriveBytes(password, saltBytes, 100000, HashAlgorithmName.SHA256);
    byte[] hash = pbkdf2.GetBytes(32); // 32 bytes for hash

    // Combine salt and hash for storage (e.g., as Base64)
    byte[] hashBytes = new byte[48];
    Array.Copy(saltBytes, 0, hashBytes, 0, 16);
    Array.Copy(hash, 0, hashBytes, 16, 32);

    salt = Convert.ToBase64String(saltBytes);
    return Convert.ToBase64String(hashBytes);
}
```

But there's a better way: **`PasswordHasher<TUser>`** from ASP.NET Core Identity, which handles salting and verification with configurable iterations.

### Example Using `PasswordHasher`

```csharp
using Microsoft.AspNetCore.Identity;

var hasher = new PasswordHasher<string>();
string password = "userPassword";
string hash = hasher.HashPassword(null, password); // returns a formatted string

// Verification
var result = hasher.VerifyHashedPassword(null, hash, password); // Success or Failed
```

`PasswordHasher` stores the hash with metadata (algorithm, iteration count, salt) in a single string, making it easy to store in a single database column.

---

## 28.4 Authentication

Authentication answers the question: **Who are you?** It's the process of verifying the identity of a user or system.

### 28.4.1 ASP.NET Core Identity

ASP.NET Core Identity is a membership system that provides user registration, login, password management, and more. It integrates with EF Core and supports external logins (Google, Facebook, etc.).

**Setup:**

Add the packages:

```bash
dotnet add package Microsoft.AspNetCore.Identity.EntityFrameworkCore
dotnet add package Microsoft.AspNetCore.Authentication.JwtBearer
```

Configure in `Program.cs`:

```csharp
builder.Services.AddDbContext<AppDbContext>(options =>
    options.UseSqlServer(connectionString));

builder.Services.AddIdentity<IdentityUser, IdentityRole>()
    .AddEntityFrameworkStores<AppDbContext>()
    .AddDefaultTokenProviders();

builder.Services.ConfigureApplicationCookie(options =>
{
    options.Cookie.HttpOnly = true;
    options.ExpireTimeSpan = TimeSpan.FromHours(8);
    options.SlidingExpiration = true;
});
```

Then you can use `SignInManager` and `UserManager` in your controllers.

### 28.4.2 JWT (JSON Web Tokens) for APIs

For stateless APIs, JWT is a popular choice. A JWT is a self‑contained token that includes claims (e.g., user ID, roles) and is digitally signed.

**JWT flow:**

1. User logs in with credentials.
2. Server validates credentials and issues a JWT.
3. Client stores the token (e.g., in local storage) and sends it in the `Authorization` header for subsequent requests.
4. Server validates the token on each request.

**Configuring JWT in ASP.NET Core:**

```csharp
builder.Services.AddAuthentication(JwtBearerDefaults.AuthenticationScheme)
    .AddJwtBearer(options =>
    {
        options.TokenValidationParameters = new TokenValidationParameters
        {
            ValidateIssuer = true,
            ValidateAudience = true,
            ValidateLifetime = true,
            ValidateIssuerSigningKey = true,
            ValidIssuer = builder.Configuration["Jwt:Issuer"],
            ValidAudience = builder.Configuration["Jwt:Audience"],
            IssuerSigningKey = new SymmetricSecurityKey(
                Encoding.UTF8.GetBytes(builder.Configuration["Jwt:Key"]))
        };
    });
```

**Issuing a token:**

```csharp
var tokenHandler = new JwtSecurityTokenHandler();
var key = Encoding.UTF8.GetBytes(_configuration["Jwt:Key"]);
var tokenDescriptor = new SecurityTokenDescriptor
{
    Subject = new ClaimsIdentity(new[] { new Claim(ClaimTypes.Name, username) }),
    Expires = DateTime.UtcNow.AddHours(1),
    Issuer = _configuration["Jwt:Issuer"],
    Audience = _configuration["Jwt:Audience"],
    SigningCredentials = new SigningCredentials(new SymmetricSecurityKey(key), SecurityAlgorithms.HmacSha256Signature)
};
var token = tokenHandler.CreateToken(tokenDescriptor);
var tokenString = tokenHandler.WriteToken(token);
```

---

## 28.5 Authorization

Authorization answers: **What are you allowed to do?** It determines whether an authenticated user has permission to access a resource.

### 28.5.1 Role‑Based Authorization

Assign roles to users and check them.

```csharp
[Authorize(Roles = "Admin")]
public IActionResult DeleteAll() { ... }
```

### 28.5.2 Policy‑Based Authorization

Policies are more flexible and can incorporate claims, roles, or custom requirements.

```csharp
builder.Services.AddAuthorization(options =>
{
    options.AddPolicy("AtLeast21", policy =>
        policy.Requirements.Add(new MinimumAgeRequirement(21)));
});
```

Implement a `AuthorizationHandler`:

```csharp
public class MinimumAgeHandler : AuthorizationHandler<MinimumAgeRequirement>
{
    protected override Task HandleRequirementAsync(AuthorizationHandlerContext context, MinimumAgeRequirement requirement)
    {
        var ageClaim = context.User.FindFirst(c => c.Type == "Age");
        if (ageClaim != null && int.Parse(ageClaim.Value) >= requirement.MinimumAge)
        {
            context.Succeed(requirement);
        }
        return Task.CompletedTask;
    }
}
```

Then apply the policy:

```csharp
[Authorize(Policy = "AtLeast21")]
public IActionResult BuyAlcohol() { ... }
```

---

## 28.6 Secure Communication (HTTPS)

Always use HTTPS in production. It encrypts data in transit, preventing eavesdropping and man‑in‑the‑middle attacks.

In ASP.NET Core, you can enforce HTTPS:

```csharp
app.UseHttpsRedirection();
```

Also, configure HSTS (HTTP Strict Transport Security) to tell browsers to always use HTTPS.

---

## 28.7 Putting It All Together: Securing the Notes API

Let's add authentication and authorization to the Notes API from Chapter 22. We'll use JWT for stateless authentication.

### Step 1: Add NuGet Packages

```bash
dotnet add package Microsoft.AspNetCore.Authentication.JwtBearer
dotnet add package Microsoft.AspNetCore.Identity.EntityFrameworkCore
```

### Step 2: Update `AppDbContext` to Support Identity

```csharp
public class AppDbContext : IdentityDbContext<IdentityUser>
{
    public AppDbContext(DbContextOptions<AppDbContext> options) : base(options) { }
    public DbSet<Note> Notes { get; set; }
}
```

### Step 3: Configure Services in `Program.cs`

```csharp
using Microsoft.AspNetCore.Identity;
using Microsoft.EntityFrameworkCore;
using Microsoft.AspNetCore.Authentication.JwtBearer;
using Microsoft.IdentityModel.Tokens;
using System.Text;
using NotesApi.Data;
using NotesApi.Models;

var builder = WebApplication.CreateBuilder(args);

// Add Identity
builder.Services.AddDbContext<AppDbContext>(options =>
    options.UseSqlite("Data Source=notes.db"));
builder.Services.AddIdentity<IdentityUser, IdentityRole>()
    .AddEntityFrameworkStores<AppDbContext>()
    .AddDefaultTokenProviders();

// Add JWT Authentication
builder.Services.AddAuthentication(options =>
{
    options.DefaultAuthenticateScheme = JwtBearerDefaults.AuthenticationScheme;
    options.DefaultChallengeScheme = JwtBearerDefaults.AuthenticationScheme;
})
.AddJwtBearer(options =>
{
    options.TokenValidationParameters = new TokenValidationParameters
    {
        ValidateIssuer = true,
        ValidateAudience = true,
        ValidateLifetime = true,
        ValidateIssuerSigningKey = true,
        ValidIssuer = builder.Configuration["Jwt:Issuer"],
        ValidAudience = builder.Configuration["Jwt:Audience"],
        IssuerSigningKey = new SymmetricSecurityKey(Encoding.UTF8.GetBytes(builder.Configuration["Jwt:Key"]))
    };
});

builder.Services.AddControllers();
builder.Services.AddEndpointsApiExplorer();
builder.Services.AddSwaggerGen();

var app = builder.Build();

// ... middleware and database creation (as before)

app.UseHttpsRedirection();
app.UseAuthentication(); // <-- Add this
app.UseAuthorization();

app.MapControllers();

// Ensure roles created (optional)
using (var scope = app.Services.CreateScope())
{
    var roleManager = scope.ServiceProvider.GetRequiredService<RoleManager<IdentityRole>>();
    if (!await roleManager.RoleExistsAsync("Admin"))
    {
        await roleManager.CreateAsync(new IdentityRole("Admin"));
    }
}

app.Run();
```

### Step 4: Add JWT Settings to `appsettings.json`

```json
{
  "Jwt": {
    "Key": "your-very-long-secret-key-here-at-least-32-characters",
    "Issuer": "NotesApi",
    "Audience": "NotesApiClient"
  }
}
```

**Important:** In production, store the key securely (e.g., Azure Key Vault, environment variables).

### Step 5: Create an Auth Controller

```csharp
[Route("api/[controller]")]
[ApiController]
public class AuthController : ControllerBase
{
    private readonly UserManager<IdentityUser> _userManager;
    private readonly SignInManager<IdentityUser> _signInManager;
    private readonly IConfiguration _configuration;

    public AuthController(UserManager<IdentityUser> userManager, SignInManager<IdentityUser> signInManager, IConfiguration configuration)
    {
        _userManager = userManager;
        _signInManager = signInManager;
        _configuration = configuration;
    }

    [HttpPost("register")]
    public async Task<IActionResult> Register([FromBody] RegisterModel model)
    {
        var user = new IdentityUser { UserName = model.Username, Email = model.Email };
        var result = await _userManager.CreateAsync(user, model.Password);
        if (result.Succeeded)
        {
            return Ok(new { message = "User created" });
        }
        return BadRequest(result.Errors);
    }

    [HttpPost("login")]
    public async Task<IActionResult> Login([FromBody] LoginModel model)
    {
        var user = await _userManager.FindByNameAsync(model.Username);
        if (user != null && await _userManager.CheckPasswordAsync(user, model.Password))
        {
            var token = GenerateJwtToken(user);
            return Ok(new { token });
        }
        return Unauthorized();
    }

    private string GenerateJwtToken(IdentityUser user)
    {
        var tokenHandler = new JwtSecurityTokenHandler();
        var key = Encoding.UTF8.GetBytes(_configuration["Jwt:Key"]);
        var tokenDescriptor = new SecurityTokenDescriptor
        {
            Subject = new ClaimsIdentity(new[]
            {
                new Claim(ClaimTypes.Name, user.UserName),
                new Claim(ClaimTypes.NameIdentifier, user.Id)
            }),
            Expires = DateTime.UtcNow.AddHours(1),
            Issuer = _configuration["Jwt:Issuer"],
            Audience = _configuration["Jwt:Audience"],
            SigningCredentials = new SigningCredentials(new SymmetricSecurityKey(key), SecurityAlgorithms.HmacSha256Signature)
        };
        var token = tokenHandler.CreateToken(tokenDescriptor);
        return tokenHandler.WriteToken(token);
    }
}

public class RegisterModel
{
    public string Username { get; set; }
    public string Email { get; set; }
    public string Password { get; set; }
}

public class LoginModel
{
    public string Username { get; set; }
    public string Password { get; set; }
}
```

### Step 6: Protect the Notes Controller

Add `[Authorize]` to the `NotesController` to require authentication.

```csharp
[Authorize]
[Route("api/[controller]")]
[ApiController]
public class NotesController : ControllerBase
{
    // ... existing code
}
```

Optionally, associate notes with the logged‑in user. You can get the username from `User.Identity.Name`.

### Step 7: Test

- Register a user: `POST /api/auth/register` with username, email, password.
- Login: `POST /api/auth/login` – get a JWT token.
- Call `GET /api/notes` with `Authorization: Bearer <token>` header.

Now your API is secured!

---

## 28.8 Best Practices

### 1. Use HTTPS Everywhere

Enforce HTTPS in production. Use HSTS.

### 2. Keep Secrets Out of Code

Use configuration, environment variables, or a secrets manager.

### 3. Hash Passwords Properly

Use a strong, slow hashing algorithm like PBKDF2, bcrypt, or Argon2. .NET's `PasswordHasher` is a good choice.

### 4. Implement Proper Authentication and Authorization

Use well‑tested libraries (ASP.NET Core Identity, JWT) rather than rolling your own.

### 5. Validate All Input

Never trust client‑side validation. Validate on the server.

### 6. Use Anti‑Forgery Tokens for State‑Changing Operations

In MVC/Razor Pages, always include anti‑forgery tokens.

### 7. Log Security Events

Log failed login attempts, password changes, and other sensitive actions. Monitor for anomalies.

### 8. Keep Dependencies Updated

Regularly update frameworks and libraries to patch known vulnerabilities.

### 9. Follow the Principle of Least Privilege

Give users and services only the permissions they need.

### 10. Conduct Security Reviews

Regularly review code and architecture for potential security issues.

---

## 28.9 Chapter Summary

In this chapter, you've learned how to secure your C# applications:

- Common vulnerabilities like SQL injection, XSS, and CSRF, and how to prevent them.
- Protecting secrets with environment variables, User Secrets, and Azure Key Vault.
- Hashing and salting passwords using `PasswordHasher`.
- Implementing authentication with ASP.NET Core Identity and JWT.
- Authorization with roles and policies.
- Securing communication with HTTPS.
- A practical example securing the Notes API.

Security is a continuous process, not a one‑time task. Stay informed about new threats and update your practices accordingly.

In the next chapter, **The Road Ahead – What's New in C#**, we'll look at recent language features and the future direction of C# and .NET.

**Exercises:**

1. Extend the Notes API to associate each note with the logged‑in user. Modify the `Note` model to include a `UserId` and adjust the controller to only return notes belonging to the current user.
2. Add a role "Admin" and create a policy that allows only admins to delete notes. Implement an admin‑only endpoint.
3. Write a unit test that verifies that an unauthorized request to a protected endpoint returns 401 Unauthorized.
4. Research and implement OAuth 2.0 with an external provider (e.g., Google) using ASP.NET Core Identity.
5. Use a tool like OWASP ZAP to scan your local Notes API for vulnerabilities and fix any findings.

Now, get ready to explore the future of C# in Chapter 29!